In [1]:
import torch
from torch import nn
import torchvision

In [2]:
Data_root = "fashionMnsit"

In [4]:
train_data = torchvision.datasets.FashionMNIST(
    root = Data_root,
    train = True,
    download = True,
    transform = torchvision.transforms.ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting fashionMnsit/FashionMNIST/raw/train-images-idx3-ubyte.gz to fashionMnsit/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting fashionMnsit/FashionMNIST/raw/train-labels-idx1-ubyte.gz to fashionMnsit/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting fashionMnsit/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to fashionMnsit/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting fashionMnsit/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to fashionMnsit/FashionMNIST/raw



/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459064158/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
test_data = torchvision.datasets.FashionMNIST(
    root = Data_root,
    train = False,
    download = True,
    transform = torchvision.transforms.ToTensor(),
)

In [8]:
Batch_size = 64

train_data_loader = torch.utils.data.DataLoader(train_data,Batch_size)
test_data_loader = torch.utils.data.DataLoader(train_data,Batch_size)

In [16]:
for X ,y in test_data_loader:
    print(X.shape)
    print(y.shape,y.dtype,torch.unique(y))
    break
    

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64 tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [51]:
class NeuralNetwork(nn.Module):
 def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.flatten()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512), # just like dense layer
        nn.ReLU(), # activation funcions
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU(),
        
    )
            
def forwerd(self,X):
    X = self.flatten(X)
    logits = self.linear_relu_stack(X)
    return logits


In [52]:
model = NeuralNetwork().to(device)
print(model)

AttributeError: module 'torch.nn' has no attribute 'Functionl'

In [29]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr =1e-3)

In [39]:
def train(dataloader, model, loss_fn, optimzer):
  size = len(dataloader.dataset)

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # calculate error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimzer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"training loss: {loss} [{current}/{size}]")

In [40]:
def test(dataloader, model):
  size = len(dataloader.dataset)
  model.eval()

  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss = test_loss +  loss_fn(pred, y).item()
      correct = correct + (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss = test_loss/size
  correct = correct/size

  print(f"Test loss: {100*correct}, average loss: {test_loss}")

In [41]:
EPOCHS = 3

for epoch in range(EPOCHS):
  print(f"EPOCH: {epoch + 1}\n-------------")

  train(train_data_loader, model, loss_fn, optimizer)
  test(test_data_loader, model)

print("Done")

EPOCH: 1
-------------
training loss: 2.3044848442077637 [0/60000]
training loss: 2.2964067459106445 [6400/60000]
training loss: 2.280771493911743 [12800/60000]
training loss: 2.2746810913085938 [19200/60000]
training loss: 2.2620155811309814 [25600/60000]
training loss: 2.2660624980926514 [32000/60000]
training loss: 2.246718645095825 [38400/60000]
training loss: 2.23829984664917 [44800/60000]
training loss: 2.234917402267456 [51200/60000]
training loss: 2.228663921356201 [57600/60000]
Test loss: 50.54833333333333, average loss: 0.03461398298740387
EPOCH: 2
-------------
training loss: 2.225874185562134 [0/60000]
training loss: 2.2041516304016113 [6400/60000]
training loss: 2.178143262863159 [12800/60000]
training loss: 2.182429790496826 [19200/60000]
training loss: 2.130647659301758 [25600/60000]
training loss: 2.168872833251953 [32000/60000]
training loss: 2.1247763633728027 [38400/60000]
training loss: 2.1124825477600098 [44800/60000]
training loss: 2.110618829727173 [51200/60000]


In [42]:
torch.save(model,"model.pth")

In [50]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

X, y = torch.tensor(test_data[5][0]).to(device), torch.tensor(test_data[0][1]).to(device)

with torch.no_grad():
  pred = model(X)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(predicted, actual)

Trouser Ankle boot


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
